In [2]:
import pandas as pd
import numpy as np
import altair as alt
import sklearn.decomposition
import sklearn.cluster
import scipy.stats

# Laden der Daten

Wir laden die Daten aus der Excel-Tabelle, die man von der Webseite herunterladen kann.

In [4]:
raw = pd.read_excel("Wahl-O-Mat Sachsen 2024_Datensatz.xlsx", sheet_name=1)
raw

,Partei: Nr.,Partei: Kurzbezeichnung,Partei: Name,These: Nr.,These: Titel,These: These,Position: Position,Position: Begründung
0,1,CDU,Christlich Demokratische Union Deutschlands,1,Braunkohleabbau,In Sachsen soll auch nach 2038 noch Braunkohle...,stimme nicht zu,Als Sächsische Union halten wir am gesetzlich ...
1,2,AfD,Alternative für Deutschland,1,Braunkohleabbau,In Sachsen soll auch nach 2038 noch Braunkohle...,stimme zu,Solange es keine konkurrenzfähige und sowohl g...
2,3,DIE LINKE,DIE LINKE,1,Braunkohleabbau,In Sachsen soll auch nach 2038 noch Braunkohle...,stimme nicht zu,Schon heute ist die Kohleverstromung in Teilen...
3,4,GRÜNE,BÜNDNIS 90/DIE GRÜNEN,1,Braunkohleabbau,In Sachsen soll auch nach 2038 noch Braunkohle...,stimme nicht zu,Der Kohleausstieg muss und wird deutlich vor 2...
4,5,SPD,Sozialdemokratische Partei Deutschlands,1,Braunkohleabbau,In Sachsen soll auch nach 2038 noch Braunkohle...,stimme nicht zu,Wir stehen zum vereinbarten Kohle-Ausstieg bis...
...,...,...,...,...,...,...,...,...
717,15,BÜNDNIS DEUTSCHLAND,BÜNDNIS DEUTSCHLAND,38,Touristische Nutzung der Sächsischen Schweiz,"Die Möglichkeiten, den Nationalpark Sächsische...",neutral,Die Tourismusförderung kann auch in diesem Geb...
718,16,BSW,Bündnis Sahra Wagenknecht - Vernunft und Gerec...,38,Touristische Nutzung der Sächsischen Schweiz,"Die Möglichkeiten, den Nationalpark Sächsische...",stimme zu,Mit seinen landschaftlich reizvollen Regionen ...
719,17,FREIE SACHSEN,FREIE SACHSEN,38,Touristische Nutzung der Sächsischen Schweiz,"Die Möglichkeiten, den Nationalpark Sächsische...",stimme zu,Die Sächsische Schweiz gehört zu unseren schön...
720,18,V-Partei³,"V-Partei³ - Partei für Veränderung, Vegetarier...",38,Touristische Nutzung der Sächsischen Schweiz,"Die Möglichkeiten, den Nationalpark Sächsische...",stimme nicht zu,Eine zunehmende touristische Nutzung kann zu e...


Die Positionen der Parteien liegen als Text vor, hier kodieren wir diese als Zahlen.

In [5]:
antworten_long = pd.DataFrame(
    {
        "Aussage": raw["These: These"],
        "Partei": raw["Partei: Kurzbezeichnung"],
        "Position": [
            {"stimme nicht zu": -1, "neutral": 0, "stimme zu": 1}[a]
            for a in raw["Position: Position"]
        ],
        "Begründung": raw["Position: Begründung"],
    }
)
antworten_long

,Aussage,Partei,Position,Begründung
0,In Sachsen soll auch nach 2038 noch Braunkohle...,CDU,-1,Als Sächsische Union halten wir am gesetzlich ...
1,In Sachsen soll auch nach 2038 noch Braunkohle...,AfD,1,Solange es keine konkurrenzfähige und sowohl g...
2,In Sachsen soll auch nach 2038 noch Braunkohle...,DIE LINKE,-1,Schon heute ist die Kohleverstromung in Teilen...
3,In Sachsen soll auch nach 2038 noch Braunkohle...,GRÜNE,-1,Der Kohleausstieg muss und wird deutlich vor 2...
4,In Sachsen soll auch nach 2038 noch Braunkohle...,SPD,-1,Wir stehen zum vereinbarten Kohle-Ausstieg bis...
...,...,...,...,...
717,"Die Möglichkeiten, den Nationalpark Sächsische...",BÜNDNIS DEUTSCHLAND,0,Die Tourismusförderung kann auch in diesem Geb...
718,"Die Möglichkeiten, den Nationalpark Sächsische...",BSW,1,Mit seinen landschaftlich reizvollen Regionen ...
719,"Die Möglichkeiten, den Nationalpark Sächsische...",FREIE SACHSEN,1,Die Sächsische Schweiz gehört zu unseren schön...
720,"Die Möglichkeiten, den Nationalpark Sächsische...",V-Partei³,-1,Eine zunehmende touristische Nutzung kann zu e...


Und dann bauen wir noch eine Pivot-Tabelle, die die Partien als Spalten und die Aussagen als Zeilen hat.

In [6]:
antworten = antworten_long.pivot(index="Aussage", columns="Partei", values="Position")
antworten

Partei,AfD,BSW,BÜNDNIS DEUTSCHLAND,BüSo,Bündnis C,CDU,DIE LINKE,Die PARTEI,FDP,FREIE SACHSEN,FREIE WÄHLER,GRÜNE,PIRATEN,SPD,TIERSCHUTZ hier!,V-Partei³,WU,dieBasis,ÖDP
Aussage,,,,,,,,,,,,,,,,,,,
Alle Geflüchteten sollen Zugang zu gebührenfreien Deutschkursen erhalten.,-1,1,-1,1,-1,1,1,1,1,-1,-1,1,1,1,1,1,-1,1,-1
Alle sächsischen Polizistinnen und Polizisten sollen im Rahmen von verpflichtenden Fortbildungen gegen Rassismus sensibilisiert werden müssen.,-1,1,-1,0,-1,-1,1,1,1,-1,-1,1,1,1,0,1,-1,0,1
An Feier- und Gedenktagen wie Karfreitag und Volkstrauertag sollen Tanzveranstaltungen weiterhin verboten sein.,-1,-1,-1,1,1,1,-1,-1,-1,1,0,0,-1,0,1,-1,1,0,1
An den Hochschulen des Freistaats soll für militärische Zwecke geforscht werden dürfen.,1,1,1,-1,-1,1,-1,-1,1,0,-1,1,-1,1,-1,-1,0,-1,-1
"An den Schulen des Freistaats sollen weiterhin Kopfnoten (Betragen, Fleiß, Mitarbeit, Ordnung) vergeben werden.",1,-1,1,1,1,1,-1,-1,1,1,-1,-1,-1,1,1,-1,1,1,-1
An den sächsischen Grenzen zu den europäischen Nachbarstaaten sollen weiterhin Personenkontrollen stattfinden.,1,1,1,1,1,1,-1,-1,-1,1,1,-1,-1,0,1,-1,1,1,-1
"Arbeitnehmerinnen und Arbeitnehmer sollen einen gesetzlichen Anspruch haben, sich für Weiterbildung freistellen zu lassen („Bildungsurlaub“).",1,1,-1,1,-1,-1,1,1,-1,0,1,1,1,1,1,1,0,1,1
Bei den Wahlen zum Sächsischen Landtag sollen Jugendliche ab 16 Jahren wählen dürfen.,-1,1,-1,-1,-1,-1,1,1,1,0,-1,1,1,1,1,1,-1,0,1
Das 5G-Mobilfunknetz soll schnellstmöglich ausgebaut werden.,0,1,1,0,-1,1,1,1,1,-1,1,1,1,1,0,1,0,-1,-1


In [7]:
anzahl_parteien = len(antworten.columns)
anzahl_fragen = len(antworten)

# Visualisierung der Positionen

Als erstes schauen wir uns die Positionen der Parteien in einer Grafik an. Hier kann man manchmal direkt schon Muster bezüglich Fragen oder Parteien erkennen.

In [8]:
chart = (
    alt.Chart(
        antworten_long,
        title="Positionen der Parteien",
        width=500,
        height=600,
    )
    .mark_rect()
    .encode(
        alt.X("Partei"),
        alt.Y("Aussage"),
        alt.Fill("Position:O", scale=alt.Scale(scheme="pinkyellowgreen")),
        [
            alt.Tooltip("Aussage"),
            alt.Tooltip("Partei"),
            alt.Tooltip("Position"),
            alt.Tooltip("Begründung"),
        ],
    )
)
chart.save("antworten.json")
chart

alt.Chart(...)

# Hauptkomponentenanalyse (PCA) bezüglich Parteien

Mit der Pivot-Tabelle können wir eine Hauptkomponentenanalyse ("principal component analysis") durchführen. Dazu betrachten wir den "Raum der Fragen", bei dem jede Frage eine Dimension ist. Jede Partei mit ihren Positionen kann dann als ein Punkt in diesem hochdimensionalen Raum aufgefasst werden. Bei den ungefähr 30 Fragen können wir uns den 30 dimensionalen Raum nicht mehr vorstellen. Die Hauptkomponentenanalyse sucht nun jene irgendwie im Raum liegende Achse, entlang derer diese Punktewolke am längsten ist. Dies ist die Achse, die am meisten erklären kann. Das ist die erste Hauptachse. Dann wird die nächste dazu rechtwinklige Achse gesucht, die am zweitmeisten erklären kann. Dies könnte man so lange machen, bis man alle 30 Dimensionen wieder hat, dann hätte man letztlich die Punktwolke nur gedreht und ein neues Koordinatensystem gefunden.

Der Trick ist aber, dass wir nach zwei Dimensionen abschneiden um eine sinnige grafische Darstellung zu bekommen. Somit bekommen wir die zwei wichtigsten Dimensionen raus, egal wie viele Fragen es gibt. Damit ist die Hauptkomponentenanalyse ein Verfahren der *Dimensionsreduktion*.

Das Ergebnis ist eine zweidimensionale Karte der Parteien. Was die beiden Achsen bedeuten, ist aber nicht so ganz zu interpretieren. Das interessante ist auf jeden Fall die Nähe zwischen Parteien und Distanz zu anderen Parteien.

In [9]:
pca = sklearn.decomposition.PCA(2)
reduced = pca.fit_transform(antworten.T)

pca_df = pd.DataFrame(
    {
        "Partei": antworten.columns,
        "Komponente 1": reduced[:, 0],
        "Komponente 2": reduced[:, 1],
    }
)
pca_df.head()

chart = (
    alt.Chart(
        pca_df,
        title="PCA der Positionen",
        width=600,
        height=400,
    )
    .mark_text()
    .encode(
        alt.X("Komponente 1"),
        alt.Y("Komponente 2"),
        alt.Text("Partei"),
    )
    .interactive()
)
chart.save("parteien_pca2.json")
chart

alt.Chart(...)

## Bedeutung der Achsen

Die Interpretation der Hauptachsen ist schwer, weil diese Achse eine bestimmte Kombination von Fragen ist. Man muss sich die Fragen anschauen, welche am meisten beitragen um zu verstehen, was die Achse sein könnte.

Wir bauen uns zuerst einen weiteren "data frame" mit den Gewichten auf und stellen diese dann grafisch dar. In der Grafik muss man dann auf jene Aussagen schauen, die ein hohes Gewicht in die eine oder andere Richtung haben. Man kann sich zum Beispiel die obersten und untersten drei Aussagen anschauen und fragen: Welche politische Richtung ist es, wenn man den ersten drei zustimmt und die letzten drei ablehnt? Was ist es, wenn man die ersten drei ablehnt und den untersten drei zustimmt? Dann hat man herausgefunden, was die Achse bedeutet.

In [10]:
hauptachsen = pd.DataFrame(
    {
        "Frage": antworten.index,
        "Gewicht 1": pca.components_[0],
        "Gewicht 2": pca.components_[1],
    }
)
hauptachsen.head()

,Frage,Gewicht 1,Gewicht 2
0,Alle Geflüchteten sollen Zugang zu gebührenfre...,0.161264,0.041016
1,Alle sächsischen Polizistinnen und Polizisten ...,0.231214,0.022458
2,An Feier- und Gedenktagen wie Karfreitag und V...,-0.109512,-0.077190
3,An den Hochschulen des Freistaats soll für mil...,-0.050875,0.296680
4,An den Schulen des Freistaats sollen weiterhin...,-0.213334,-0.016393


In [11]:
chart = (
    alt.Chart(
        hauptachsen,
        title="Hauptachse 1",
    )
    .mark_bar()
    .encode(alt.X("Gewicht 1"), alt.Y("Frage", sort="-x"), alt.Tooltip("Frage"))
)
chart.save("hauptachse_1_fragen.json")
chart

alt.Chart(...)

Das gleiche kann man dann mit der zweiten Achse machen. Diese ist in der Regel deutlich schwerer zu interpretieren.

In [12]:
chart = (
    alt.Chart(
        hauptachsen,
        title="Hauptachse 2",
    )
    .mark_bar()
    .encode(alt.X("Gewicht 2"), alt.Y("Frage", sort="-x"), alt.Tooltip("Frage"))
)
chart.save("hauptachse_2_fragen.json")
chart

alt.Chart(...)

# Korrelationen der Parteien

Wir können nun eine Sortierung der Parteien entlang der ersten Hauptkomponente aufstellen, die wir im weiteren Verlauf nutzen werden um die Korrelationsdiagramme vorzusortieren.

In [13]:
parteien_order = list(pca_df.sort_values("Komponente 1")["Partei"])
parteien_order

['Bündnis C',
 'WU',
 'BÜNDNIS DEUTSCHLAND',
 'AfD',
 'FREIE SACHSEN',
 'FREIE WÄHLER',
 'CDU',
 'BüSo',
 'TIERSCHUTZ hier!',
 'dieBasis',
 'FDP',
 'BSW',
 'V-Partei³',
 'SPD',
 'ÖDP',
 'GRÜNE',
 'Die PARTEI',
 'PIRATEN',
 'DIE LINKE']

Und damit erstellen wir ein Korrelationsdiagramm bei denen die einzelnen Fragen als Beobachtungen genommen werden. Man kann also sehen, welche Parteien ähnlich antworten wie andere Parteien. Durch die Sortierung nach der ersten Hauptachse bilden sich meist Blöcke.

Haben zwei Parteien eine Korrelation von >0,7 so haben sie gewisse Parallelen. Bei <-0,7 sind die Parteien ziemlich entgegengesetzt. Dazwischen sind sie weder ähnlich noch gegensätzlich sondern eher unabhängig voneinander.

In [14]:
p = scipy.stats.spearmanr(antworten)
df = pd.DataFrame(p.correlation)
df.index = antworten.columns
df.columns = antworten.columns
long = df.melt(ignore_index=False)
long.index.name = "Partei 1"
long.columns = ["Partei 2", "Korrelation"]
long = long.reset_index()

chart = (
    alt.Chart(long, title="Korrelation der Parteien")
    .mark_rect()
    .encode(
        alt.X("Partei 1", sort=parteien_order),
        alt.Y("Partei 2", sort=parteien_order),
        alt.Fill(
            "Korrelation", scale=alt.Scale(scheme="pinkyellowgreen", domain=(-1, 1))
        ),
        [alt.Tooltip("Partei 1"), alt.Tooltip("Partei 2"), alt.Tooltip("Korrelation")],
    )
)
chart.save("korrelation_parteien.json")
chart

alt.Chart(...)

# Cluster-Analyse mit K-Means

Dann können wir noch in den 30 dimensionalen Raum gehen und dort nach Häufungen suchen. Mit dem K-Means-Algorithmus teilen wir den Raum in K Teile auf. Man kann das K wählen und so steuern, wie viele Gruppen man findet. Hier ist das einmal für einige Werte von K ausgeführt und die Parteien innerhalb jeder Gruppe aufgelistet.

Es beantwortet also die Frage: Wenn wir alle Parteien in K Gruppen aufteilen, welche Gruppen erhalten wir dann?

In [15]:
for k in [2, 3, 4, 5, 10]:
    k_means = sklearn.cluster.KMeans(n_clusters=k, random_state=0).fit(antworten.T)
    parteien_df = pd.DataFrame(
        {"Name": antworten.columns, "Cluster": k_means.labels_}
    ).set_index("Name")

    print(f"{k = }:")
    for key, values in parteien_df.groupby("Cluster").groups.items():
        print("-", ", ".join(sorted(values, key=lambda x: x.lower())))
    print()

k = 2:
- AfD, Bündnis C, BÜNDNIS DEUTSCHLAND, BüSo, CDU, dieBasis, FDP, FREIE SACHSEN, FREIE WÄHLER, TIERSCHUTZ hier!, WU
- BSW, DIE LINKE, Die PARTEI, GRÜNE, PIRATEN, SPD, V-Partei³, ÖDP

k = 3:
- AfD, Bündnis C, BÜNDNIS DEUTSCHLAND, BüSo, CDU, dieBasis, FREIE SACHSEN, FREIE WÄHLER, TIERSCHUTZ hier!, WU
- BSW, FDP, SPD, ÖDP
- DIE LINKE, Die PARTEI, GRÜNE, PIRATEN, V-Partei³

k = 4:
- AfD, Bündnis C, BÜNDNIS DEUTSCHLAND, CDU, FREIE WÄHLER, WU
- BSW, FDP, SPD, ÖDP
- DIE LINKE, Die PARTEI, GRÜNE, PIRATEN, V-Partei³
- BüSo, dieBasis, FREIE SACHSEN, TIERSCHUTZ hier!

k = 5:
- AfD, Bündnis C, BÜNDNIS DEUTSCHLAND, FREIE WÄHLER, WU
- BSW, SPD, ÖDP
- DIE LINKE, Die PARTEI, GRÜNE, PIRATEN, V-Partei³
- BüSo, dieBasis, FREIE SACHSEN, TIERSCHUTZ hier!
- CDU, FDP

k = 10:
- FREIE WÄHLER
- SPD, ÖDP
- BüSo, dieBasis, TIERSCHUTZ hier!
- CDU
- Bündnis C, WU
- V-Partei³
- DIE LINKE, Die PARTEI, GRÜNE, PIRATEN
- BSW
- FDP
- AfD, BÜNDNIS DEUTSCHLAND, FREIE SACHSEN



# Analyse der Fragen

Man kann die Analyse auch herumdrehen und von den Fragen her anschauen. Dann sind die Parteien die Datenpunkte. Wir können eine Hauptkomponentenanalyse der Fragen im Raum der Parteien durchführen. Dann sieht man, welche Fragen in ähnliche Richtungen gehen, weil die gleichen Parteien sie ähnlich beantwortet haben. Da einige Fragen positiv oder negativ gestellt haben, findet man negativ gestellte Fragen punktgespiegelt auf der anderen Seite des Diagrams.

In [16]:
pca = sklearn.decomposition.PCA(2)
reduced = pca.fit_transform(antworten)

pca_df = pd.DataFrame(
    {
        "Frage": antworten.index,
        "Komponente 1": reduced[:, 0],
        "Komponente 2": reduced[:, 1],
    }
)
pca_df.head()

chart = (
    alt.Chart(
        pca_df,
        title="PCA der Fragen",
        width=600,
        height=400,
    )
    .mark_circle()
    .encode(
        alt.X("Komponente 1"),
        alt.Y("Komponente 2"),
        alt.Tooltip("Frage"),
    )
    .interactive()
)
chart.save("fragen_pca2.json")
chart

alt.Chart(...)

## Bedeutung der Achsen

Auch hier können wir uns anschauen, welche Parteien vor allem die Richtungen vorgeben.

In [17]:
hauptachsen = pd.DataFrame(
    {
        "Frage": antworten.columns,
        "Gewicht 1": pca.components_[0],
        "Gewicht 2": pca.components_[1],
    }
)
hauptachsen.head()

,Frage,Gewicht 1,Gewicht 2
0,AfD,0.266030,0.239501
1,BSW,-0.048412,0.290187
2,BÜNDNIS DEUTSCHLAND,0.276790,0.005437
3,BüSo,0.162264,0.284203
4,Bündnis C,0.337132,0.043083


In [18]:
chart = (
    alt.Chart(
        hauptachsen,
        title="Hauptachse 1",
    )
    .mark_bar()
    .encode(alt.X("Gewicht 1"), alt.Y("Frage", sort="-x"), alt.Tooltip("Frage"))
)
chart.save("hauptachse_1_parteien.json")
chart

alt.Chart(...)

In [19]:
chart = (
    alt.Chart(
        hauptachsen,
        title="Hauptachse 2",
    )
    .mark_bar()
    .encode(alt.X("Gewicht 2"), alt.Y("Frage", sort="-x"), alt.Tooltip("Frage"))
)
chart.save("hauptachse_2_parteien.json")
chart

alt.Chart(...)

## Korrelation der Fragen

Interessant ist auch eine Korrelation der Fragen. Eine hohe positive Korrelation zeigt, dass die meisten Parteien auf beide Fragen gleich antworten. Eine sehr negative Korrelation zeigt, dass diese Fragen meist gegensätzlich beantwortet werden. Manche Korrelationen erscheinen offensichtlich und sind daher wenig informativ. Andere Korrelationen sind jedoch ziemlich überraschend.

In [20]:
p = scipy.stats.spearmanr(antworten.T)
df = pd.DataFrame(p.correlation)
df.index = antworten.index
df.columns = antworten.index
long = df.melt(ignore_index=False)
long.index.name = "Aussage 1"
long.columns = ["Aussage 2", "Korrelation"]
long = long.reset_index()

fragen_order = list(pca_df.sort_values("Komponente 1")["Frage"])

chart = (
    alt.Chart(long, title="Korrelation der Fragen")
    .mark_rect()
    .encode(
        alt.X("Aussage 1", sort=fragen_order),
        alt.Y("Aussage 2", sort=fragen_order),
        alt.Fill(
            "Korrelation", scale=alt.Scale(scheme="pinkyellowgreen", domain=(-1, 1))
        ),
        [
            alt.Tooltip("Aussage 1"),
            alt.Tooltip("Aussage 2"),
            alt.Tooltip("Korrelation"),
        ],
    )
)
chart.save("korrelation_fragen.json")
chart

alt.Chart(...)

# Cluster der Fragen

Wir können die Fragen auch noch in fünf Gruppen einteilen. Hier ist es allerdings etwas blöd, dass einige Fragen negativ gestellt sind und daher in einen anderen Cluster kommen. Trotzdem kann man noch ein bisschen etwas erkennen.

In [21]:
k_means = sklearn.cluster.KMeans(n_clusters=5, random_state=0).fit(antworten)
parteien_df = pd.DataFrame(
    {"Name": antworten.index, "Cluster": k_means.labels_}
).set_index("Name")

for key, values in parteien_df.groupby("Cluster").groups.items():
    print("1.")
    print()
    for frage in sorted(values):
        print("    -", frage)
    print()

1.

    - An den Hochschulen des Freistaats soll für militärische Zwecke geforscht werden dürfen.
    - Der Freistaat soll sich dafür einsetzen, dass gentechnisch veränderte Pflanzen in Sachsen angebaut werden dürfen.
    - Die Mietpreisbremse in Leipzig und Dresden soll abgeschafft werden.
    - Die Videoüberwachung an öffentlichen Plätzen soll ausgeweitet werden.

1.

    - Alle Geflüchteten sollen Zugang zu gebührenfreien Deutschkursen erhalten.
    - Alle sächsischen Polizistinnen und Polizisten sollen im Rahmen von verpflichtenden Fortbildungen gegen Rassismus sensibilisiert werden müssen.  
    - Das 5G-Mobilfunknetz soll schnellstmöglich ausgebaut werden.
    - Das sächsische Landesamt für Verfassungsschutz soll gestärkt werden.
    - In Sachsen sollen weitere Windkraftanlagen gebaut werden.
    - Sachsen soll sich dafür einsetzen, dass Deutschland weiterhin Waffen an die Ukraine liefert.

1.

    - Bei den Wahlen zum Sächsischen Landtag sollen Jugendliche ab 16 Jahren wählen dü